**Quantitative Value Strategy**

"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

In [4]:
!sudo pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter
import math 
from scipy import stats 




**Importing Our List of Stocks & API Token**



In [6]:
stocks = pd.read_csv('/content/sp_500_stocks.csv')
stocks= stocks.drop(labels=[135,219,467,484],axis=0)
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [7]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'


**Making Our First API Call**

It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).


In [8]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data


{'avgTotalVolume': 77598166,
 'calculationPrice': 'close',
 'change': 2.38,
 'changePercent': 0.01566,
 'close': 162.45,
 'closeSource': 'iaflcfio',
 'closeTime': 1686784089334,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 157.45,
 'delayedPriceTime': 1737525711543,
 'extendedChange': -0.49,
 'extendedChangePercent': -0.00326,
 'extendedPrice': 157.96,
 'extendedPriceTime': 1670194464480,
 'high': 162.45,
 'highSource': 'ilm5i eutr apyene dcd1e',
 'highTime': 1737793423861,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 159.99,
 'iexCloseTime': 1698587618589,
 'iexLastUpdated': 1712269508949,
 'iexMarketPercent': 0.017445632878704927,
 'iexOpen': 156.417,
 'iexOpenTime': 1716962102966,
 'iexRealtimePrice': 159.61,
 'iexRealtimeSize': 50,
 'iexVolume': 1126556,
 'isUSMarketOpen': False,
 'lastTradeTime': 1714137945806,
 'latestPrice': 161.82,
 'latestSource': 'Close',
 'latestTime': 'July 21, 2022',
 'latestUpdate': 1694414920


**Parsing Our API Call**

This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:


In [9]:
pe_ratio = data['peRatio']
pe_ratio



26.23


**Executing A Batch API Call & Building Our DataFrame**
it's now time to execute several batch API calls and add the information we need to our DataFrame
 More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.


In [10]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']


In [11]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,130.28,30.74,N/A
1,AAL,14.24,-3.95,N/A
2,AAP,195.16,21.57,N/A
3,AAPL,160.09,26.01,N/A
4,ABBV,150.62,21.53,N/A
...,...,...,...,...
496,YUM,121.05,22.4,N/A
497,ZBH,113.22,109.33,N/A
498,ZBRA,334.41,22.38,N/A
499,ZION,53.34,5.72,N/A



**Removing Glamour Stocks**

The opposite of a "value stock" is a "glamour stock".

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.


In [12]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,GOOGL,118.58,1.07,N/A
1,NRG,36.22,2.3,N/A
2,EBAY,47.96,2.62,N/A
3,AMZN,128.48,3.0,N/A
4,COF,115.57,4.61,N/A
5,SYF,33.94,4.63,N/A
6,AIG,53.66,4.66,N/A
7,PVH,65.43,4.7,N/A
8,KSS,31.40,4.74,N/A
9,F,13.00,4.75,N/A



**Calculating the Number of Shares to Buy**

We now need to calculate the number of shares we need to buy.

To do this, we will use the portfolio_input function .

I have included this function below.


In [13]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")


In [14]:
portfolio_input()


Enter the value of your portfolio:1000000


we can now use the global portfolio_size variable to calculate the number of shares that our strategy should purchase.

In [15]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,GOOGL,118.58,1.07,168
1,NRG,36.22,2.3,552
2,EBAY,47.96,2.62,417
3,AMZN,128.48,3.0,155
4,COF,115.57,4.61,173
5,SYF,33.94,4.63,589
6,AIG,53.66,4.66,372
7,PVH,65.43,4.7,305
8,KSS,31.40,4.74,636
9,F,13.00,4.75,1538



Building a Better (and More Realistic) Value Strategy

Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a composite basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

    Price-to-earnings ratio
    Price-to-book ratio
    Price-to-sales ratio
    Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
    Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.



In [16]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit


Let's move on to building our DataFrame. we'll notice that I use the abbreviation rv often. It stands for robust value, which is what we'll call this sophisticated strategy moving forward.

In [17]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

In [18]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,131.92,N/A,32.18,N/A,7.12,N/A,5.74,N/A,21.099445,N/A,10.751826,N/A,N/A
1,AAL,14.40,N/A,-3.87,N/A,-1.13,N/A,0.2915,N/A,-22.592307,N/A,1.019608,N/A,N/A
2,AAP,190.93,N/A,21.52,N/A,4.04,N/A,1.07,N/A,11.898627,N/A,2.594451,N/A,N/A
3,AAPL,158.47,N/A,25.34,N/A,37.64,N/A,6.59,N/A,19.993615,N/A,15.501894,N/A,N/A
4,ABBV,150.01,N/A,21.29,N/A,16.22,N/A,4.7,N/A,11.819642,N/A,8.155258,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,122.87,N/A,21.92,N/A,-4.24,N/A,5.32,N/A,19.853893,N/A,8.928230,N/A,N/A
497,ZBH,111.02,N/A,112.26,N/A,1.91,N/A,2.99,N/A,18.445637,N/A,5.458429,N/A,N/A
498,ZBRA,345.44,N/A,21.96,N/A,6.06,N/A,3.05,N/A,16.734931,N/A,6.928088,N/A,N/A
499,ZION,53.15,N/A,5.68,N/A,1.42,N/A,2.4,N/A,4.512960,N/A,2.255275,N/A,N/A



**Dealing With Missing Data in Our DataFrame**

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using.

You can use pandas' isnull method to identify missing data:


In [19]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,188.500,N/A,60.81,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,294.660,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,289.170,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,97.970,N/A,50.17,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.78,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,25.270,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
164,ETFC,50.040,N/A,14.74,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
185,FLIR,58.230,N/A,32.17,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
189,FOX,32.550,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRT,107.400,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


There are two main approaches:

    Drop missing data from the data set (pandas' dropna method is useful here)
    Replace missing data with a new value (pandas' fillna method is useful here)

we will replace missing data with the average non-NaN data point from that column.


In [20]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)


In [21]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score



Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

    Price-to-earnings ratio
    Price-to-book ratio
    Price-to-sales ratio
    EV/EBITDA
    EV/GP

Here's how we'll do this:


In [22]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe


0      0.744511
1      0.031936
2       0.51497
3      0.592814
4      0.510978
         ...   
496     0.52495
497    0.986028
498    0.526946
499    0.085828
500    0.868263
Name: PE Percentile, Length: 501, dtype: object
0      0.808383
1       0.05988
2      0.632735
3       0.97006
4      0.924152
         ...   
496    0.055888
497    0.279441
498    0.758483
499    0.178643
500     0.94012
Name: PB Percentile, Length: 501, dtype: object
0      0.822355
1      0.015968
2      0.161677
3      0.850299
4      0.762475
         ...   
496    0.800399
497    0.536926
498    0.543912
499    0.434132
500     0.94511
Name: PS Percentile, Length: 501, dtype: object
0      0.800399
1      0.007984
2       0.41517
3      0.776447
4      0.411178
         ...   
496    0.770459
497    0.718563
498    0.670659
499     0.07984
500    0.932136
Name: EV/EBITDA Percentile, Length: 501, dtype: object
0      0.800399
1      0.041916
2      0.153693
3       0.92016
4      0.654691
         ...   
4

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,131.92,N/A,32.18,0.744511,7.12,0.808383,5.7400,0.822355,21.099445,0.800399,10.751826,0.800399,N/A
1,AAL,14.40,N/A,-3.87,0.031936,-1.13,0.05988,0.2915,0.015968,-22.592307,0.007984,1.019608,0.041916,N/A
2,AAP,190.93,N/A,21.52,0.51497,4.04,0.632735,1.0700,0.161677,11.898627,0.41517,2.594451,0.153693,N/A
3,AAPL,158.47,N/A,25.34,0.592814,37.64,0.97006,6.5900,0.850299,19.993615,0.776447,15.501894,0.92016,N/A
4,ABBV,150.01,N/A,21.29,0.510978,16.22,0.924152,4.7000,0.762475,11.819642,0.411178,8.155258,0.654691,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,122.87,N/A,21.92,0.52495,-4.24,0.055888,5.3200,0.800399,19.853893,0.770459,8.928230,0.702595,N/A
497,ZBH,111.02,N/A,112.26,0.986028,1.91,0.279441,2.9900,0.536926,18.445637,0.718563,5.458429,0.409182,N/A
498,ZBRA,345.44,N/A,21.96,0.526946,6.06,0.758483,3.0500,0.543912,16.734931,0.670659,6.928088,0.520958,N/A
499,ZION,53.15,N/A,5.68,0.085828,1.42,0.178643,2.4000,0.434132,4.512960,0.07984,2.255275,0.135729,N/A



**Calculating the RV Score**

We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.


In [23]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,131.92,N/A,32.18,0.744511,7.12,0.808383,5.7400,0.822355,21.099445,0.800399,10.751826,0.800399,0.79521
1,AAL,14.40,N/A,-3.87,0.031936,-1.13,0.05988,0.2915,0.015968,-22.592307,0.007984,1.019608,0.041916,0.031537
2,AAP,190.93,N/A,21.52,0.51497,4.04,0.632735,1.0700,0.161677,11.898627,0.41517,2.594451,0.153693,0.375649
3,AAPL,158.47,N/A,25.34,0.592814,37.64,0.97006,6.5900,0.850299,19.993615,0.776447,15.501894,0.92016,0.821956
4,ABBV,150.01,N/A,21.29,0.510978,16.22,0.924152,4.7000,0.762475,11.819642,0.411178,8.155258,0.654691,0.652695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,122.87,N/A,21.92,0.52495,-4.24,0.055888,5.3200,0.800399,19.853893,0.770459,8.928230,0.702595,0.570858
497,ZBH,111.02,N/A,112.26,0.986028,1.91,0.279441,2.9900,0.536926,18.445637,0.718563,5.458429,0.409182,0.586028
498,ZBRA,345.44,N/A,21.96,0.526946,6.06,0.758483,3.0500,0.543912,16.734931,0.670659,6.928088,0.520958,0.604192
499,ZION,53.15,N/A,5.68,0.085828,1.42,0.178643,2.4000,0.434132,4.512960,0.07984,2.255275,0.135729,0.182834



**Selecting the 50 Best Value Stocks¶**

we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.


In [24]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,GOOGL,119.20,N/A,1.07,0.043912,0.3120,0.061876,0.2991,0.017964,-0.452034,0.01996,-0.281616,0.001996,0.029142
1,AAL,14.40,N/A,-3.87,0.031936,-1.1300,0.05988,0.2915,0.015968,-22.592307,0.007984,1.019608,0.041916,0.031537
2,KSS,32.10,N/A,4.77,0.063872,0.8941,0.093812,0.2126,0.007984,3.512268,0.03992,1.030696,0.045908,0.050299
3,AIG,52.71,N/A,4.63,0.057884,0.7558,0.077844,0.7717,0.107784,2.790341,0.027944,0.775792,0.021956,0.058683
4,UNM,33.29,N/A,7.44,0.115768,0.6304,0.071856,0.5700,0.067864,3.781367,0.047904,0.553391,0.005988,0.061876
5,GPS,9.44,N/A,-50.30,0.00998,1.4400,0.183633,0.2215,0.00998,4.990559,0.097804,0.729263,0.017964,0.063872
6,GM,35.81,N/A,6.14,0.091816,0.8370,0.083832,0.4033,0.033932,1.887139,0.021956,1.627977,0.093812,0.06507
7,PVH,63.00,N/A,4.50,0.052894,0.8326,0.081836,0.4768,0.0499,4.771481,0.08982,1.080900,0.053892,0.065669
8,LNC,53.00,N/A,7.48,0.11976,0.5998,0.067864,0.4569,0.043912,5.373816,0.107784,0.442612,0.003992,0.068663
9,PRU,96.72,N/A,7.74,0.127745,0.8506,0.087824,0.5502,0.065868,4.994054,0.0998,0.553873,0.007984,0.077844



**Calculating the Number of Shares to Buy**

We'll use the portfolio_input function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.


In [25]:
portfolio_input()

Enter the value of your portfolio:10000000


In [26]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,GOOGL,119.20,1677,1.07,0.043912,0.3120,0.061876,0.2991,0.017964,-0.452034,0.01996,-0.281616,0.001996,0.029142
1,AAL,14.40,13888,-3.87,0.031936,-1.1300,0.05988,0.2915,0.015968,-22.592307,0.007984,1.019608,0.041916,0.031537
2,KSS,32.10,6230,4.77,0.063872,0.8941,0.093812,0.2126,0.007984,3.512268,0.03992,1.030696,0.045908,0.050299
3,AIG,52.71,3794,4.63,0.057884,0.7558,0.077844,0.7717,0.107784,2.790341,0.027944,0.775792,0.021956,0.058683
4,UNM,33.29,6007,7.44,0.115768,0.6304,0.071856,0.5700,0.067864,3.781367,0.047904,0.553391,0.005988,0.061876
5,GPS,9.44,21186,-50.30,0.00998,1.4400,0.183633,0.2215,0.00998,4.990559,0.097804,0.729263,0.017964,0.063872
6,GM,35.81,5585,6.14,0.091816,0.8370,0.083832,0.4033,0.033932,1.887139,0.021956,1.627977,0.093812,0.06507
7,PVH,63.00,3174,4.50,0.052894,0.8326,0.081836,0.4768,0.0499,4.771481,0.08982,1.080900,0.053892,0.065669
8,LNC,53.00,3773,7.48,0.11976,0.5998,0.067864,0.4569,0.043912,5.373816,0.107784,0.442612,0.003992,0.068663
9,PRU,96.72,2067,7.74,0.127745,0.8506,0.087824,0.5502,0.065868,4.994054,0.0998,0.553873,0.007984,0.077844



**Formatting Our Excel Output**

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. 

In [27]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)



**Creating the Formats We'll Need For Our .xlsx File**

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

    String format for tickers
    $XX.XX format for stock prices
    $XX,XXX format for market capitalization
    Integer format for the number of shares to purchase
    Float formats with 1 decimal for each valuation metric



In [28]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [29]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])



**Saving Our Excel Output**

In [30]:
writer.save()